# Lab 4-2: Databases, Custom Functions, and the Million-Song Dataset

## The Million-Song Dataset

The [Million-Song Dataset](https://labrosa.ee.columbia.edu/millionsong/) is a collection of metadata and audio features for a million pop songs from the past 50+ years. It's a whopping 280GB to download! But there are a variety of subsets -- and joins with other data sources like user-generated genre tags from Tagtraum and MusicBrainz -- available from [their website](https://labrosa.ee.columbia.edu/millionsong/pages/getting-dataset#subset). It's a great place to start when exploring musical metadata and what goes into an audience's conception of genre.

We'll start by looking at one of the smaller files from that dataset: `artist_term.db`. This is an SQLite database with multiple tables, containing information about user-generated genre tags applied to artists in the Million-Song Dataset (MSD, from now on). Here are the names of the 5 tables contained in that database.

Import the requisite libraries, and create a SQL database engine for `artist_term.db`.

- Import `pandas` as `pd` and `create_engine` from `sqlalchemy`.
- Create a database engine called `engine` from the SQLite database `artist_term.db`.
- Print the table names from that database.

<hr />


## Some Good References

Here is a helpful [Pandas cheatsheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) to remind you of some of the basic commands and workflows we'll be exploring here. Another great reference for pertinent skills and terms is introduced in [Python Data Science Toolbox, Part 1](https://campus.datacamp.com/courses/python-data-science-toolbox-part-1/) and and [Importing Data in Python, Part 1](https://campus.datacamp.com/courses/importing-data-in-python-part-1). Please do feel free to review the slides and exercises there as you go through this project. You can also see some great student projects on this [class blog](https://medium.com/modeling-music).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

import pandas as pd
from sqlalchemy import create_engine

query = "SELECT * FROM artist_mbtag"

# Create engine
engine = create_engine('sqlite:///artist_term.db')


df = pd.read_sql_query(query, engine.connect())

# Print the table name
print(df)


                artist_id                 mbtag
0      AR002UA1187B9A637D                    uk
1      AR002UA1187B9A637D                  rock
2      AR002UA1187B9A637D           garage rock
3      AR006821187FB5192B                  bass
4      AR00A6H1187FB5402A               detroit
...                   ...                   ...
24772  ARZZMWO1187B9AFB36                   usa
24773  ARZZMWO1187B9AFB36                 metal
24774  ARZZXT51187FB4627E  classic pop and rock
24775  ARZZYRB1187B99D0B6                  jazz
24776  ARZZYRB1187B99D0B6              canadian

[24777 rows x 2 columns]


## Unpacking genre tags

Now let's dig in. The three tables `artists`, `mbtags`, and `terms` contain lists of elements contained in the first two tables, `artist_mbtag` and `artist_term`. We'll dig into the Musicbrainz genre tags linked to artists in the `artist_mbtag` table. Let's now load and display the first few rows of the `artist_mbtag` table.

- Use `read_sql_query()` to select *all* records from the `artist_mbtag` table and store the results in a dataframe of the same name.
- Print the first 5 rows of `artist_mbtag`.

<hr />

Be sure to double-check the documentation for `read_sql_query()` (or your notes).

In [7]:
# Read in genre tags from the `artist_mbtag` table in the database
artist_mbtag = pd.read_sql_query("SELECT * FROM artist_mbtag", engine.connect())

# Display the first 5 records of `artist_mbtag`
artist_mbtag.head(5)

,artist_id,mbtag
0,AR002UA1187B9A637D,uk
1,AR002UA1187B9A637D,rock
2,AR002UA1187B9A637D,garage rock
3,AR006821187FB5192B,bass
4,AR00A6H1187FB5402A,detroit


## Finding artist names

There is some interesting data in here but those `artist_id` labels aren't very helpful. Thankfully, MSD has another file that matches the unique artist IDs to the artist's names.

Import `unique_artists.txt` into a dataframe `artist_meta` and display the first few rows.

- Use `read_csv()` to import `unique_artists.txt` into a dataframe named `artist_meta`.
- [According to MSD](https://labrosa.ee.columbia.edu/millionsong/pages/getting-dataset#subset), the fields in this file are separated by the string `<SEP>`. The file has no header, and we only need the first column (artist ID) and the fourth column (artist name) for our analysis so please set those parameters accordingly.
- Rename the columns to `artist_id` and `artist_name`, respectively.

<hr />

Remember that `.columns` takes a list. And it will be clear later (if not already) why we need to rename `artist_id` in particular.

Also, note that because of the `<` and `>` in the delimiter, Python may produce a warning message. Don't worry about that, as long as it also produces the expected output. (Which is... -- remember: always evaluate your output before going on, making sure it matches what you predicted or expected, or if you didn't know what to expect, that it makes sense in context once you see it.)

In [10]:
# Load a dataframe that connects artist IDs with artist names, and rename columns
artist_meta = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/ML Class/Week 4/Song lab/unique_artists.txt/unique_artists.txt', sep='<SEP>', header=None, usecols=[0,3])
artist_meta.columns = ['artist_id', 'artist_name']

# Display the first 5 rows of 'artist_meta'
artist_meta.head()

<ipython-input-10-4d3dc0af2c97>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  artist_meta = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/ML Class/Week 4/Song lab/unique_artists.txt/unique_artists.txt', sep='<SEP>', header=None, usecols=[0,3])


,artist_id,artist_name
0,AR002UA1187B9A637D,The Bristols
1,AR003FB1187B994355,The Feds
2,AR006821187FB5192B,Stephen Varcoe/Choir of King's College_ Cambri...
3,AR009211187B989185,Carroll Thompson
4,AR009SZ1187B9A73F4,Gorodisch


## Merging artist names with their genre tags

Now that we've loaded the SQLite database with artist tags and the `<SEP>`-delimited text file with artist names, we need to join them together to make them useful. Here's what it looks like when we connect artists with the various genres users have attributed to their music.

Join the `artist_mbtag` and `artist_meta` dataframes together, creating a new dataframe `artists` that links each band name with each genre tag users associated with it.

- Use `pd.merge()` to join the two dataframes together. This is a new function but a really important one (see below). The first two arguments should be the names of the two dataframes to join. The `how=` argument specifies the kind of join -- in our case, `inner`.
- Specify the field to merge by with the `on=` argument: `artist_id`. (This is why we renamed the column when we imported it!)
- Remove the no-longer-needed column `artist_id` by subsetting `artists` to include only the `artist_name` and `mbtag` columns.
- Display the head of `artists` to confirm a successful merge.

<hr />

Were the two tables we're interested in part of the same database, we could use `INNER JOIN` directly in SQL. However, because they are in different formats, and we've now imported them both into `pandas` dataframes, we can use the `pandas` function `merge()` to do the same thing.

We'll just stick with `inner` joins for now, but we'll go over a deeper explanation of different join types later in the course.

In [12]:
# Merge artist tags with artist names, joining on the common field 'artist_id'
artists = pd.merge(artist_mbtag, artist_meta, how='inner', on='artist_id')

# Clean artists, removing the no longer needed column 'artist_id'
artists = artists[['artist_name', 'mbtag']]

# Display the first 5 rows of the new, much more readable dataframe
artists.head()

,artist_name,mbtag
0,The Bristols,uk
1,The Bristols,rock
2,The Bristols,garage rock
3,Stephen Varcoe/Choir of King's College_ Cambri...,bass
4,The Meatmen,detroit


## Genre tags by band

Let's look up some of our favorite artists, and see which genres they are associated with in MSD and Musicbrainz! Let's create a custom function `tags_for_band()` that returns a list of genre tags for a specified artist.

- This function takes two arguments: `dataframe` and `band`. Assuming a user provides the dataframe `artists`, define `tags` so that it returns a list of all the tags associated with the specified `band`.
- Test it out with a few different artist names, and see if the results make sense to you. One example in the dataset is provided.

In [58]:
def tags_for_band(dataframe, band):
    """From a MSD dataframe and a band name, print and return a list of tags for that band."""
    tags=(dataframe[dataframe['artist_name'] == band]['mbtag'].tolist())
    return tags

tags_for_band(artists, 'David Bowie')
#tags_for_band(artists, 'The Beatles')
#tags_for_band(artists, 'The Rolling Stones')

['british',
 'uk',
 'rock',
 'experimental',
 'glam rock',
 'english',
 'classic pop and rock',
 'post-disco',
 'britannique',
 'soul',
 'blue-eyed soul',
 'pop',
 'art rock']

## Bands by genre tag

Now let's look up some of our favorite genres and see what artists are represented. Using the same pattern as in the previous task, create a new custom function that returns all the bands associated with a given genre tag.

In [59]:
def bands_for_tag(dataframe, tag):
    """From a MSD dataframe and a genre tag, print and return a list of bands associated with that tag."""

    bands=(dataframe[dataframe['mbtag'] == tag]['artist_name'].tolist())
    return bands

bands_for_tag(artists, 'glam rock')

['Sparks',
 'Alice Cooper',
 'Iggy Pop',
 'Goldfrapp',
 'Whitesnake',
 'Simple Kid',
 'Sweet',
 'David Bowie',
 'Queen',
 'Brian May',
 'Lou Reed',
 'Smokie',
 'Wizzard',
 'Elton John',
 'Scissor Sisters',
 'Marc Bolan / T. Rex']

## The most represented artists and genres

Now that we've explored some of our favorites, let's see which artists and which genre tags are most common in this dataset. Let's find, and then display, the number of occurrences of each artist name and each genre tag in the `artists` dataframe.

- Define `top_tags` and `top_artists` using the `.value_counts()` method learned in the Ramen lab.

In [60]:
# Define the most common tags and artists in the dataset
top_tags =artists['mbtag'].value_counts()
top_artists = artists['artist_name'].value_counts()
# Print the results
print(top_tags)
print(top_artists)

mbtag
classic pop and rock    1073
american                1027
uk                      1013
british                  975
rock and indie           920
                        ... 
rottun                     1
dubtronica                 1
serbian                    1
roger waters               1
slam                       1
Name: count, Length: 2321, dtype: int64
artist_name
Moby                  22
Indidginus            20
Yodelice              20
Musetta               20
Prodigy               20
                      ..
Gayle San              1
Alek Szahala           1
I Haunt Wizards        1
Orchestra Baobab       1
Scouting for Girls     1
Name: count, Length: 8797, dtype: int64


## The tags of genre-transcending artists

Some artists have more genre tags than others. In some cases, this is a reflection of popularity -- more popularity leads to more listeners adding their two cents about genre to a crowd-sourced database. In some cases, this is a reflection of an artist's ability to bridge or transcend genre boundaries.

Either way, let's see what genre tags are most common for the artists with the longest list of genre associations (20 or more). Maybe that will tell us something about their music -- or about music in general, and the relative rigidity of some genres. Let's find the genre tags most associated with the 12 artists most heavily tagged in the corpus (those with 20 genre tags or more).

- Import the `Counter` function from the package `collections` (these are case-sensitive)
- Create an empty list `tags_of_popular_bands`.
- Loop through the 12 bands at the beginning of `top_artists`. (Note the code used here, as you'll need it later -- why do you think it might be necessary?)
- Find the tags associated with each band (there's a function for that!) and add them to `tags_for_popular_bands`.
- Use the `Counter()` function to count the number of occurrences of each tag in `tags_for_popular_bands`. Print the results.

In [68]:
# Import Counter from collections
from collections import Counter

# Create an empty list tags_of_popular_bands
tags_of_popular_bands =[]

#print(top_artists.index.values[:12])
# Loop through the 12 most common artists
for band in top_artists.index.values[:12]:

    i = tags_for_band(artists, band)
    # Find the tags represented by those bands and add them to tags_of_popular_bands\
    tags_of_popular_bands.extend(i)


print(tags_of_popular_bands)
print(len(tags_of_popular_bands))
a = Counter(tags_of_popular_bands)
# Print a count of tags associated with the 12 most common artists
print(a)

['rap', 'electronic', 'american', 'electronic', 'ambient', 'house', 'downtempo', 'techno', 'electronica', 'harlem', 'darien', 'dance and electronica', 'united states', 'producteur', 'producer', 'compositeur', 'composer', 'parolier', 'rock', 'lyricist', 'rave', 'pop', 'psychedelic trance', 'world fusion', 'downtempo', 'didgeridoo', 'idm', 'organica', 'indidginus', 'electronic music', 'organic soundscapes', 'didg', 'music', 'electronica', 'chillout', 'downbeat', 'psybient', 'psytrance', 'progressive psytrance', 'dubstep', 'slide didgeridoo', 'slide didg', 'folk', 'compositeur', 'france', 'french', 'producer', 'composer', 'producteur', 'français', 'guitarist', 'guitariste', 'bassiste', 'chanteur', 'francophone', 'singer', 'keyboardist', 'bassist', 'claviériste', 'parolier', 'lyricist', 'arrangeur', 'tripop', 'nu-jazz', 'italian', 'nu jazz', 'audial crack', 'chill-out', 'electronica', 'easy listening', 'female vocalist', 'female voices', 'instrumental', 'female vocals', 'female', 'electro 

## Genre pairings

That's not particularly instructive, so let's look at what genres are paired with each other throughout the entire dataset! To begin, here is every unique pair of genres associated with The Beatles.

Define a function `get_tag_pairs()` that returns a list of all unique pairs of genres associated with a given band. Print the result for The Beatles.

- This function takes two arguments: `dataframe` and `band`. Assuming a user provides the dataframe `artists`, define `tags` so that it returns a list of all the tags associated with the specified `band`. (This should sound familiar)
- Complete the `if` statement so that it is only true when the two tags are different, and in alphabetical order. (This will ensure that we don't count `rock`-`rock` as a "pair", and that `rock`-`pop` and `pop`-`rock` are not considered *different* pairs.)
- Inside the `if` statement, create a string that concatenates `tag` with `tag2`, with ` & ` in between, and add that string to the `tag_pairs` list.
- Return `tag_pairs`.

<hr />

The looping syntax is provided because it's somewhat complex but please make sure you understand it before you go on to the next task.

In [74]:
def get_tag_pairs(dataframe, band):
    """Return list of pairs of tags associated with the same band."""

    tags = tags_for_band(dataframe, band)
    tag_pairs = []
    for tag in tags:
        for tag2 in tags:
            if tag != tag2 and tag < tag2:
                tag_pairs.append(tag + '&' + tag2)

    return tag_pairs

print(get_tag_pairs(artists, 'The Beatles'))

['british&rock', 'british&pop', 'british&british invasion', 'british&heavy metal', 'british&classical pop', 'british&instrumental pop', 'british&folk-rock', 'british&pop rock', 'british&orchestral', 'british&psychedelic', 'british&merseybeat', 'british&rock roll', 'british&tribute albums', 'british&british psychedelia', 'british&orchestral pop', 'british&singer songwriter', 'british&indie rock', 'rock&rock roll', 'rock&tribute albums', 'rock&singer songwriter', 'pop&rock', 'pop&pop rock', 'pop&psychedelic', 'pop&rock roll', 'pop&tribute albums', 'pop&singer songwriter', 'british invasion&rock', 'british invasion&pop', 'british invasion&heavy metal', 'british invasion&classical pop', 'british invasion&instrumental pop', 'british invasion&folk-rock', 'british invasion&pop rock', 'british invasion&orchestral', 'british invasion&psychedelic', 'british invasion&merseybeat', 'british invasion&rock roll', 'british invasion&tribute albums', 'british invasion&british psychedelia', 'british inva

## Genre pairings - the whole kit and caboodle

That's a lot of genre pairings just for one band! Ironically, it's simpler to look at the genre pairings for the whole dataset, and get a handle on which ones tend to go together most frequently. Again, this is a result of both the relationship between the two genres *and* the relative popularity of both genre tags.

With that context in mind, let's take a look!  Let's use the new `get_tag_pairs()` function on all artists in the dataset, and display the most common genre tag pairings in descending order.

- Use `set()` and `list()` together to reduce `artists['artist_name']` to a list of unique artist names, each occurring only once.
- Define an empty list `all_tag_pairs`.
- While looping through the unique band names in `artist_names`, use `get_tag_pairs()` to retrieve all genre tag pairings for each band and add them to `all_tag_pairs`.
- Use `Counter()` to count the number of occurrences of each genre pairing in `all_tag_pairs`, and assign it to a Pandas series `tag_pair_count`.
- Display `tag_pair_count`. Apply the method `.sort_values()` with the argument `ascending=False` to display them in descending order from the most to the least common.

<hr />

There are a couple of new things here. First, we're taking the dictionary that `Counter()` produces and assigning it to a `pandas` series. This will allow us both to display it in a more user-friendly way, and to more easily sort it so our output is more meaningful to read.

Also, the `.sort_values()` function allows us to take that `pandas` series and sort it with a single, easy-to-use method. This is another one worth adding to your notes, even though we haven't met it in detail before.

In [79]:
# Get a list of unique artist names
artist_names = list(set(artists['artist_name']))

# Define an empty list `all_tag_pairs`
all_tag_pairs = []

# Loop through the bands in artist_names
for band in artist_names:
    # Get all tag pairs for each artist and adding them to all_tag_pairs
    all_tag_pairs.extend(get_tag_pairs(artists, band))


# Count the occurrences of each genre tag pair in all_tag_pairs, and assign the results to a pandas series
tag_pair_count = pd.Series(Counter(all_tag_pairs)).sort_values(ascending=False)

# Display the results
print(tag_pair_count)



british&uk                    752
british&english               608
english&uk                    593
american&rock                 312
classic pop and rock&uk       245
                             ... 
opera&rock                      1
américain&ingénieur du son      1
américain&sound engineer        1
opera&psychedelic rock          1
electronica&punk                1
Length: 19829, dtype: int64


## Conclusion

Looks like American and British/English music tops the charts in this dataset and is strongly associated with rock, pop, punk, and indie music. It would be interesting to tease out all of the UK- and US-specific terms and make a more direct comparison -- are some genres more "English-like" (punk or classic rock, perhaps?) and others more "American-like" (alternative or jazz, maybe?). We'll save that for a future project!